In [1]:
from ast import literal_eval

import pandas as pd
from tqdm import tqdm
import dotenv
import argparse
from configs import Config, create_rag_config
from rag_modules import create_chain, get_retriever, read_csv_for_rag_query, set_columms_from_config
from rag_modules import combined_key_query_builder
from utils import check_valid_score, format_docs, record_right_answer, set_seed

/data/ephemeral/canola/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dotenv.load_dotenv()

set_seed(42)

data_path = "data/test_v1.0.2.csv"

In [4]:
retriever = get_retriever(embedding_model="BAAI/bge-m3", k=5)

# config 내 query builder type을 통해 query로 사용할 columns setting
columns = set_columms_from_config(1)
query_builder = combined_key_query_builder(columns)

# CSV 파일 로드
df = read_csv_for_rag_query(file_path=data_path)

# 결과 저장용 리스트
results = []

# Inference
for _, row in tqdm(df[:2].iterrows(), total=len(df), desc="Processing rows"):

    question_plus_string = ("\n\n<보기>:\n" + row["question_plus"]) if row["question_plus"] else ""
    question = row["question"] + question_plus_string
    choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(row["choices"])])

    query = query_builder.build(row)

    retrieved_docs = retriever.invoke(query)

    support = format_docs(retrieved_docs)
    results.append(support)

print(results)

Connected to Pinecone index: sling-chunk


Processing rows:   0%|          | 2/869 [00:02<17:29,  1.21s/it]

["작품에는 따끔한 질타나 든든한 격려의 글을 남기기도 한다. 실제로 웹툰 작가들은 댓글을 통해 독자의 피드백을 받는다. 댓글을 통한 디지털 서사의 형성은 곧 '작가->독자'의 일방향 소통에서 '독자<->독자', '작가<->독자'의 다방향 소통으로의 전환이라고 할 수 있다.\n\n미술감상은 감상물이 지니고 있는 미를 맛보고자 함을 목표로 하고 있다.  감상물은 한없이 존재하고 있으나 이를 감상물로 택하지 않으면 노방의 돌에 지나지 않는다. 물론 감상자가 어떤 계기가 주어져 감상물과의 마음의 균형이 생겨야만 비로소 '감상'이 성립될 수 있음은 말할 나위조차 없다.  교사와 학생과의 예를 든다면 아무리 교사의 마음에 드는 작품이라 할지라도 학생들에게는 아무런 관심이나 감동도 불러일으키지 못하는 경우가 많다.  교사는 학생들로 하여금 감상물에 대해서 흥미를 갖도록 해야 거기에 감상 교육의 의의가 생겨난다. 2세된 유아조차도 기차나 자동차에는 흥미가 쏠려 밖으로 나가 그것을 보고 싶다고 보챈다. 우리는 여기서 감상의 싹을 엿볼 수 있다.  감상에는 감상물에 대하여 감동을 느낄 수 있는 우수한 감수성과, 청결하고 예리한 감각의 육성에 있음은 말할 필요조차 없으나 거기에 지각이나 지성이 결합되어 발달되어 나간다는 사실을 잊어서는 안 된다.  감상활동을 통해서 지각이나\n\n토론(討論, debate)은 의사결정 집단에서 어떠한 관심 주제에 대하여 의견이 다른 상대방과 번갈아가며 공평하게 의사소통하도록 구조화된 사건이다. 상호 소통적인 논증 방식으로 논리적 논증보다 더 큰 범위의 논증에 속한다.\n하지만 많은 사람들이 토론을 자신의 관점을 관철하고자 하는 사람들이 자신의 공격적인 주장을 두고 벌이는 경쟁으로 인식하고 있는데 이는 잘못된 이해다. 토론은 자신의 논증을 비판적 시각으로 검토하도록 돕고, 새로운 생각을 탐색하게 고무하는 일에 더 가깝다. 쟁점을 비판적으로 검토하고자 하는 사람들 간의 자연스럽고 신나는 상호작용이며, 토론은 새로운 세계, 새로운 기능, 새로운 생각을

In [ ]:
len(results)